<a href="https://colab.research.google.com/github/jonatasbx/Curso-CDIA/blob/main/tratamento_sus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Notebook para análise do banco de dados do SIH-SUS.

# Para a análise pretendida, a base foi filtrada com base na variável 'espec' = 2

In [2]:
import pandas as pd


In [3]:
base = pd.read_csv('https://github.com/jonatasbx/Curso-CDIA/blob/859df07e039bce314e3ac9b6233711385bd84a7c/base_tratada_pretos.xlsx')
base.info()

ParserError: Error tokenizing data. C error: Expected 1 fields in line 42, saw 38
